### Ejercicio 5: Introducción a la optimización

In [ ]:
from IPython.display import Image 
Image(filename='img_enun/E4.png')

#### Importamos Pyomo

In [ ]:
from pyomo.environ import *
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_excel('Datos/Datos_Ejercicio4.xlsx')
df

#### Creamos un problema concreto, es decir, un objeto de la clase problema que ya contenga parámetros

In [ ]:
model = ConcreteModel()

#### Necesitamos utilizar sets

In [ ]:
alumnos=np.arange(30)
asignaturas=np.arange(7)


In [ ]:
model.alum=Set(initialize=alumnos)
model.asig=Set(initialize=asignaturas)


#### Definimos las variables

In [ ]:
model.NotaAlum=Var(model.alum,model.asig,within=PositiveReals)


#### Definimos la función objetivo

In [ ]:
model.obj = Objective(expr = sum(model.NotaAlum[a,b] for a in model.alum for b in model.asig), sense=maximize)

#### Definimos la restricciones

In [ ]:
model.maxnota = ConstraintList()
for a in model.alum:
    for b in model.asig:
        model.maxnota.add(
            model.NotaAlum[a,b]<=df.Nota_max[b]
        )

In [ ]:
model.medianota = ConstraintList()
for b in model.asig:
    model.medianota.add(
        sum(model.NotaAlum[a,b] for a in model.alum)/30<=df.Nota_media[b]
    )

In [ ]:
model.minnotaalum= ConstraintList()
for a in model.alum:
    for b in model.asig:
        model.minnotaalum.add(
            model.NotaAlum[a,b]>=5
        )

#### Resolvemos el problema

In [ ]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()   

#### Exportamos a Excel

In [ ]:
df2=pd.DataFrame.from_dict(model.NotaAlum.extract_values(),orient='index')
df2.to_excel('Resultados/Result_Ej4.xlsx')


In [ ]:
print(asignaturas)

In [ ]:
notas=df2.to_numpy()
import matplotlib.pyplot as plt
plt.plot(notas,'o',color='red')
plt.xlabel('Alumno')
plt.ylabel('Nota en asignatura')

for i in asignaturas:
    plt.show()
    plt.plot(notas[29*(i):29*(i+1)],'o-')
    plt.xlabel('Alumno')
    plt.ylabel('Nota en asignatura')